<a href="https://colab.research.google.com/github/assuncaomarcos/classroom_extensions/blob/main/notebooks/MongoDB_Shell_Magics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MongoDB Shell Magics
-------

This notebook demonstrates the MongoDB Shell magics (`%mongo_config` and `%%mongo`). The magic `%%mongo` has a straightforward execution. It creates a process running the `mongosh` and writes to its standard input, displaying the result of the execution in the cell. The magic `%mongo_config` is used to configure the `mongosh` parameters to avoid entering them every time `%%mongo` is used.

## Installing MongoDB and Sample Databases
--------------

The following cell will install MongoDB, the Mongo Shell and import a few sample databases.

In [ ]:
""" Install MongoDB and import sample databases """

import subprocess
from os import path
import os
import glob


SOFTWARE_DESC = {
    "mongodb": "MongoDB",
    "mongodb_shell": "MongoDB Shell"
}

INSTALL_CMDS = {
    "mongodb": [
        "apt update",
        "apt install mongodb",
        "service mongodb start"
    ],
    "mongodb_shell": [
        "wget -qO- https://www.mongodb.org/static/pgp/server-6.0.asc | sudo tee /etc/apt/trusted.gpg.d/server-6.0.asc",
        "sudo apt-get install gnupg",
        "echo 'deb [ arch=amd64,arm64 ] https://repo.mongodb.org/apt/ubuntu focal/mongodb-org/6.0 multiverse' | "
        "sudo tee /etc/apt/sources.list.d/mongodb-org-6.0.list",
        "sudo apt update",
        "sudo apt install -y mongodb-mongosh"
    ]
}

SAMPLE_DBS_URL = "https://github.com/neelabalan/mongodb-sample-dataset.git"


def exec_cmd(command: str) -> None:
    """
    Execute a command and print error, if occurs
    :param command: the command to execute
    :return: None
    """
    try:
        subprocess.check_output(
            f"{command} > /dev/null",
            shell=True, stderr=subprocess.STDOUT
        )
    except (subprocess.CalledProcessError, Exception) as e:
        raise RuntimeError(f"Error with: {command}\n: {e}")


def install_software(software: str) -> None:
    """ Installs a given software """
    description = SOFTWARE_DESC.get(software)
    commands = INSTALL_CMDS.get(software)

    print(f"Installing {description}...")
    try:
        for cmd in commands:
            exec_cmd(cmd)
        print(f"{description} is installed.")
    except RuntimeError as e:
        print(f"Error installing {description}: {e}")


def import_sample_datasets():
    """ Clones the git repository with multiple sample datasets and imports them """
    local_clone = "sample_dbs"
    print("Cloning git repository with the sample datasets...")
    clone_path = path.join(os.getcwd(), local_clone)
    try:
        if not path.exists(clone_path):
            exec_cmd(f"git clone {SAMPLE_DBS_URL} {local_clone}")
        else:
            print("Skipping git clone as local repository seems to exist.")
            datasets = [f for f in os.listdir(local_clone)
                        if not path.isfile(path.join(local_clone, f))]
            for dataset in datasets:
                dataset_path = path.join(clone_path, dataset)
                print(f"Importing dataset {dataset}...")
                for json_file in glob.glob(f"{dataset_path}/*.json"):
                    collection = path.splitext(path.basename(json_file))[0]
                    cmd = f"mongoimport --drop --host localhost --port 27017 " \
                          f"--db {dataset} --collection {collection} --file {json_file}"
                    exec_cmd(cmd)
        print("Finished importing the sample datasets.")
    except RuntimeError as e:
        print(f"Error importing sample databases: {e}")


install_software("mongodb")
install_software("mongodb_shell")
import_sample_datasets()


## Installing the Classroom Extensions
------------------

We will install the extensions via pip+github:

In [ ]:
!pip3 install git+https://github.com/assuncaomarcos/classroom_extensions.git

Once the extensions are installed, you can load the `mongodb` extension to create the `%%mongo` and `%mongo_config` magics:

In [3]:
%load_ext classroom_extensions.mongodb

## Running MongoDB Shell Commands
------------------

In [7]:
%%mongo
show databases;

test> admin               32.00 KiB
config              12.00 KiB
local               32.00 KiB
sample_airbnb        8.00 KiB
sample_analytics    24.00 KiB
sample_geospatial    1.45 MiB
sample_mflix        40.00 KiB
sample_supplies      1.26 MiB
sample_training     64.41 MiB
sample_weatherdata   5.23 MiB
test> 

In [8]:
%%mongo
use sample_geospatial;
show collections;

test> switched to db sample_geospatial
sample_geospatial> shipwrecks
sample_geospatial> 

In [9]:
%%mongo
use sample_geospatial;
db.shipwrecks.findOne({});

test> switched to db sample_geospatial
sample_geospatial> {
  _id: ObjectId("578f6fa2df35c7fbdbaed8c5"),
  recrd: '',
  vesslterms: '',
  feature_type: 'Wrecks - Visible',
  chart: 'US,U1,graph,DNC H1409860',
  latdec: { '$numberDouble': '9.3340302' },
  londec: { '$numberDouble': '-79.9357223' },
  gp_quality: '',
  depth: '',
  sounding_type: '',
  history: '',
  quasou: '',
  watlev: 'always dry',
  coordinates: [
    { '$numberDouble': '-79.9357223' },
    { '$numberDouble': '9.3340302' }
  ]
}
sample_geospatial> 